In [19]:
import praw
import pandas as pd
import json
from tqdm.notebook import tqdm

client_id = 'QXgnNilxAJoowxnJqYSUmA'
client_secret = 'oigCTLqOddP8NkRg_CHbFDI0SElr3w'

reddit = praw.Reddit(client_id = client_id,
                     client_secret = client_secret,
                     user_agent = 'IS 596')

In [20]:
subreddit_list = ["conservative", "conspiracy", "politics", "worldnews"]

In [21]:
def subreddit_crawler(sub, limit = 10):
    subreddit = reddit.subreddit(sub)
    post_list = []
    submission_fields = ['id', 'author', 'title', 'selftext', 'num_comments',
              'score', 'upvote_ratio', 'created_utc', 'url']
    for submission in subreddit.top(limit = limit, time_filter = "year"):
        to_dict = vars(submission)
        sub_dict = {field:to_dict[field] for field in submission_fields}
        post_list.append(sub_dict)
    
    subs_data = pd.DataFrame(data = post_list, columns=submission_fields)
    file_name = "data/" + "submission_" + sub + ".csv"
    subs_data.to_csv(file_name, index = False)

In [22]:
def data_processing(sub, field = "submission"):
    file_name = "data/" + field + "_" + sub + ".csv"
    df = pd.read_csv(file_name,index_col=0)
    print("subreddit : " + sub)
    print(df.describe())
    print(df.info())

In [23]:
def comment_parser(sub, checkpoint = 0):
    file_name = "data/" + "submission_" + sub + ".csv"
    df = pd.read_csv(file_name)
    
    sublist = df["id"].tolist()
    sublist = sublist[checkpoint:]
    fields = ['id', 'author', 'body',  'parent_id',
              'score', 'subreddit', 'subreddit_id', 'submission_id']
    parsing_fields = ['id', 'author', 'body',  'parent_id',
              'score', 'subreddit', 'subreddit_id']
    
    file_name = "data/" + "comment_" + sub + ".csv"
    
    if not checkpoint:
        df = pd.DataFrame(columns=fields)
        df.to_csv(file_name)
    
    for post in tqdm(sublist):
        comment_list = []
        submission = reddit.submission(post)
        submission.comments.replace_more(limit=8)
        for comment in submission.comments.list():
            to_dict = vars(comment)
            sub_dict = {field:to_dict[field] for field in parsing_fields}
            sub_dict.update({'submission' : submission.id})
            comment_list.append(sub_dict)
        df = pd.DataFrame(data = comment_list)
        df.to_csv(file_name, mode='a', header = False)
            
    comm_data = pd.DataFrame(data = comment_list, columns=fields)
    file_name = "data/" + sub + "_comment.csv"
    subs_data.to_csv(file_name, index = False)

In [24]:
for sub in tqdm(subreddit_list):
    subreddit_crawler(sub, limit = 50)

  0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
for sub in subreddit_list:
    data_processing(sub)

subreddit : conservative
       num_comments         score  upvote_ratio   created_utc
count     50.000000     50.000000     50.000000  5.000000e+01
mean    1507.200000  10062.960000      0.766200  1.652674e+09
std     1775.331518   7250.946722      0.079383  1.005257e+07
min       17.000000   4955.000000      0.600000  1.638890e+09
25%      430.000000   5456.000000      0.710000  1.644462e+09
50%      731.000000   6667.000000      0.770000  1.652030e+09
75%     1953.000000  12102.750000      0.820000  1.660673e+09
max     8606.000000  39524.000000      0.910000  1.668478e+09
<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, ux33k8 to wf89ml
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   author        47 non-null     object 
 1   title         50 non-null     object 
 2   selftext      2 non-null      object 
 3   num_comments  50 non-null     int64  
 4   score         50 non-null     int64  
 5   upvot

In [26]:
for sub in tqdm(subreddit_list):
    comment_parser(sub, checkpoint = 0)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

NameError: name 'subs_data' is not defined